# Using MonoTools fit

Here we will go through a few examples of how to fit a mono (one transit) or duo (two transit) transiting planet system using MonoTools.

This will cover:
* Accessing lightcurve & stellar parameter data
* Fitting duotransits and monotransits by marginalising over discrete parameter spaces in multiple parameters
* Fitting such systems using the in-built margnial velocity interpolator (fastest method)
* Fitting systems which also have RVs
* Plotting the outputs

In [2]:
import numpy as np
import pandas as plt
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

from MonoTools import fit, tools, starpars

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


All data is stored in the following path which you can update in the commandline using $MONOTOOLSPATH

In [3]:
tools.MonoData_savepath

'/Volumes/LUVOIR/MonoToolsData'

### Loading Lightcurves & Stellar Params

Lightcurves are a dictionary with a few important arrays - `time`, `flux`, `flux_err` as well as:
* `cadence` - in the form "[mission letter][cadence in nearest minute]", e.g. T2 (for 2-minute TESS data), or K30 (for 30-min Kepler or K2 data). This should ideally be in an array form with matching length to `time`
* `mask` - in the form of a boolean, where those True points are the cadences to be used.
* `jd_base` - zero-point of time in BJD

However, you don't need to worry about this, because you can load lightcurves from all surveys using: